In [1]:
from v04lib import *

In [2]:
class Loading(object):
    pass

In [3]:
class LoadSet(Loading):
    
    LOADSET = {}
    
    @classmethod
    def getLS(cls,lsname):
        if lsname in cls.LOADSET:
            return cls.LOADSET[lsname]
        new = cls(lsname)
        cls.LOADSET[lsname] = new
        return new
    
    def __init__(self,lsname):
        self.lsname = lsname
        self.node_loads = {}
        self.member_loads = {}

In [4]:
class LoadCombination(Loading):
    pass

In [5]:
class NodeLoad(object):
    pass

In [6]:
import numpy as np
        
def zerofef():
    return np.mat(np.zeros((6,1)))

def makefef(l):
    return np.mat(np.array(l).reshape((6,1)))

In [7]:
class MemberLoad(object):
    
    EPSILON = 1.0E-6
    
    @classmethod
    def all_subclasses(cls):
        all_subclasses = []
    
        for subclass in cls.__subclasses__():
            all_subclasses.append(subclass)
            all_subclasses.extend(subclass.all_subclasses())

        return all_subclasses

    @classmethod
    def subclass(cls,ltype,L,data):
        classes = cls.all_subclasses()
        for c in classes:
            if c.__name__ == ltype:
                aspec = getArgSpec(c.__init__)
                args = aspec.args[2:]
                defaults = aspec.defaults or ()
                argl = zip(args,(None,)*(len(args)-len(defaults)) + defaults)
                argv = {}
                for k,v in argl:
                    if k in data and not isnan(data[k]) and data[k] is not None:
                        argv[k] = data[k]
                    elif v is not None:
                        argv[k] = v
                ##print c,args,argl,'argv =',argv
                return c(L,**argv)
        raise Exception('Invalid load type: {}'.format(ltype))
        
    def fefs(self):
        """Return the complete set of 6 fixed end forces produced by the load."""
        raise NotImplementedError()
        
    def shear(self,x):
        """Return the shear force that is in equilibrium with that
        produced by the portion of the load to the left of the point at 
        distance 'x'.  'x' may be a scalar or a 1-diemnsiional array
        of values."""
        raise NotImplementedError()
        
    def moment(self,x):
        """Return the bending moment that is in equilibrium with that
        produced by the portion of the load to the left of the point at 
        distance 'x'.  'x' may be a scalar or a 1-diemnsiional array
        of values."""
        raise NotImplementedError()

In [8]:
class UDL(MemberLoad):
    
    def __init__(self,L,W1):
        self.L = L
        self.W = W1
        
    def __repr__(self):
        return '{}(L={},W1={})'.format(self.__class__.__name__,self.L,self.W)
    
    def fefs(self):
        L = self.L
        w = self.W
        return makefef([0.,-w*L/2., -w*L*L/12., 0., -w*L/2., w*L*L/12.])
    
    def shear(self,x):
        l = x*(x>0.)*(x<=self.L) + self.L*(x>self.L)    # length of loaded portion
        return -(l*self.W)
    
    def moment(self,x):
        l = x*(x>0.)*(x<=self.L) + self.L*(x>self.L)   # length of loaded portion
        d = (x-self.L)*(x>self.L)   # distance from loaded portion to x: 0 if x <= L else x-L
        return self.W*l*(l/2.+d)
    
    @property
    def vpts(self):
        return (0.,self.L,1)
    
    @property
    def mpts(self):
        return (0.,self.L,2)

In [9]:
class LVL(MemberLoad):
    
    def __init__(self,L,W1,W2=None,A=None,B=None,C=None):
        if A is not None and B is not None and C is not None and L != (A+B+C):
            raise Exception('Cannot specify all of A, B & C')
        if A is None:
            if B is not None and C is not None:
                A = L - (B+C)
            else:
                A = 0.
        if C is None:
            if B is not None:
                C = L - (A+B)
            else:
                C = 0.
        if B is None:
            B = L - (A+C)
        if W2 is None:
            W2 = W1
        self.L = L
        self.W1 = W1
        self.W2 = W2
        self.A = A
        self.B = B
        self.C = C
        
    def fefs(self):
        """This mess was generated via sympy.  See:
        ../../examples/cive3203-notebooks/FEM-2-Partial-lvl.ipynb """
        L = float(self.L)
        a = self.A
        b = self.B
        c = self.C
        w1 = self.W1
        w2 = self.W2
        m2 = -b*(15*a*b**2*w1 + 5*a*b**2*w2 + 40*a*b*c*w1 + 20*a*b*c*w2 + 30*a*c**2*w1 + 30*a*c**2*w2 + 3*b**3*w1 + 2*b**3*w2 + 10*b**2*c*w1 + 10*b**2*c*w2 + 10*b*c**2*w1 + 20*b*c**2*w2)/(60.*(a + b + c)**2)
        m5 = b*(20*a**2*b*w1 + 10*a**2*b*w2 + 30*a**2*c*w1 + 30*a**2*c*w2 + 10*a*b**2*w1 + 10*a*b**2*w2 + 20*a*b*c*w1 + 40*a*b*c*w2 + 2*b**3*w1 + 3*b**3*w2 + 5*b**2*c*w1 + 15*b**2*c*w2)/(60.*(a + b + c)**2)
        v4 = -(b*w1*(a + b/2.) + b*(a + 2*b/3.)*(-w1 + w2)/2. + m2 + m5)/L
        v1 = -b*(w1 + w2)/2. - v4
        return makefef([0.,v1,m2,0.,v4,m5])
    
    def __repr__(self):
        return '{}(L={},W1={},W2={},A={},B={},C={})'\
               .format(self.__class__.__name__,self.L,self.W1,self.W2,self.A,self.B,self.C)
        
    def shear(self,x):
        c = (x>self.A+self.B)    # 1 if x > A+B else 0
        l = (x-self.A)*(x>self.A)*(1.-c) + self.B*c  # length of load portion to the left of x
        return -(self.W1 + (self.W2-self.W1)*(l/self.B)/2.)*l        
        
    def moment(self,x):
        c = (x>self.A+self.B)    # 1 if x > A+B else 0
        #           note: ~c doesn't work if x is scalar, thus we use 1-c
        l = (x-self.A)*(x>self.A)*(1.-c) + self.B*c  # length of load portion to the left of x
        d = (x-(self.A+self.B))*c                  # distance from right end of load portion to x
        return ((self.W1*(d+l/2.)) + (self.W2-self.W1)*(l/self.B)*(d+l/3.)/2.)*l
    
    @property
    def vpts(self):
        return (self.A,self.A+self.B,1 if self.w1==self.w2 else 2)
    
    @property
    def mpts(self):
        return (self.A,self.A+self.B,2 if self.w1==self.w2 else 3)

In [10]:
class PL(MemberLoad):
    
    def __init__(self,L,W1,A):
        self.L = L
        self.P = W1
        self.A = A
        
    def fefs(self):
        P = self.P
        L = float(self.L)
        a = self.A
        b = L-a
        m2 = -P*a*b*b/(L*L)
        m5 = P*a*a*b/(L*L)
        v1 = (m2 + m5 - P*b)/L
        v4 = -(m2 + m5 + P*a)/L
        return makefef([0.,v1,m2,0.,v4,m5])
    
    def shear(self,x):
        return -self.P*(x>self.A)
    
    def moment(self,x):
        return self.P*(x-self.A)*(x>self.A)
    
    @property
    def vpts(self):
        return (self.A-self.EPSILON,self.A+self.EPSILON,0)
    
    @property
    def mpts(self):
        return (None,self.A,1)
    
    def __repr__(self):
        return '{}(L={},W1={},A={})'.format(self.__class__.__name__,self.L,self.P,self.A)

In [11]:
class CM(MemberLoad):
    
    def __init__(self,L,W1,A):
        self.L = L
        self.M = W1
        self.A = A
        
    def fefs(self):
        L = float(self.L)
        A = self.A
        B = L - A
        M = self.M
        m2 = B*(2.*A - B)*M/L**2
        m5 = A*(2.*B - A)*M/L**2
        v1 = (M + m2 + m5)/L
        v4 = -v1
        return makefef([0,v1,m2,0,v4,m5])
    
    def shear(self,x):
        return x*0.
    
    def moment(self,x):
        return -self.M*(x>self.A)
    
    @property
    def vpts(self):
        return (None,None,0)
    
    @property
    def mpts(self):
        return (self.A-self.EPSILON,self.A+self.EPSILON,1)
    
    def __repr__(self):
        return '{}(L={},W1={},A={})'.format(self.__class__.__name__,self.L,self.M,self.A)